In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

# Data Description

The Auto Loan Dataset comes from [this Kaggle link](https://www.kaggle.com/datasets/saurabhbagchi/dish-network-hackathon?resource=download)

While this comes with a data dictionary to describe each of the variables in the training and test datasets, some of the descriptions are insufficient. As such, we will be limiting our data to variables that are easily interpretable and relevant to the project

We will be keeping 19 input columns, and 1 output column:
- ID
- Client_Income (Annual income in $)
- Car_Owned (If they own a different car prior to applying for an auto loan)
- Bike_Owned (If they own a bike prior to applying for an auto loan)
- Active_Loan (If they have any other active loan prior to applying for auto loan)
- House_Own
- Child_Count
- Credit_Amount (Principal Amount for the auto loan)
- Loan_Annuity
- Client_Income_Type (What type of industry they work in)
    - Note: We keep this column as opposed to Client_Occupation, which is more specific and has more na values
- Client_Education
- Client_Marital_Status
- Loan_Contract_Type (Cash Loan or Revolving Loan)
- Population_Region_Relative (The population of the area the person lives in)
- Age_Days
- Employed_Days
- Application_Process_Day (Day of the week the loan was applied for)
- Application_Process_Hour (Hour of the day the loan was applied for)
- Default (Did they default on their loan)

In [2]:
#Build out columns we are going to select from the dataset
id_column = ['ID']
numeric_columns = ['Client_Income', 'Child_Count', 'Credit_Amount', 'Loan_Annuity', 'Age_Days', 'Employed_Days']
binary_columns = ['Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own', 'Loan_Contract_Type', 'Default']
categorical_columns = ['Client_Income_Type', 'Client_Education', 'Client_Marital_Status']

selected_columns = id_column + numeric_columns + binary_columns + categorical_columns

selected_columns = ['ID',
 'Client_Income',
 'Car_Owned',
 'Bike_Owned',
 'Active_Loan',
 'House_Own',
 'Child_Count',
 'Credit_Amount',
 'Loan_Annuity',
 'Client_Income_Type',
 'Client_Education',
 'Client_Marital_Status',
 'Loan_Contract_Type',
 'Age_Days',
 'Employed_Days',
 'Default']

In [3]:
#Reads in the given training and testing dataset (we'll split them back into training and testing later)
#We also clean some null data by setting these cases to None
df_auto_loan_train = pd.read_csv('Train_Dataset.csv')
df_auto_loan_test = pd.read_csv('Test_Dataset.csv')
df_auto_loan = pd.concat([df_auto_loan_train, df_auto_loan_test]).replace('$', None).loc[:, selected_columns]
df_auto_loan = df_auto_loan.replace(np.nan, None).replace('#VALUE!', None)
df_auto_loan['Age_Days'] = df_auto_loan['Age_Days'].replace('x', None)

df_auto_loan

/var/folders/w5/srvyynsx0xl0xdcy3blxc3800000gp/T/ipykernel_5197/3016704985.py:3: DtypeWarning: Columns (1,7,8,16,17,18,19,20,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_auto_loan_train = pd.read_csv('Train_Dataset.csv')
/var/folders/w5/srvyynsx0xl0xdcy3blxc3800000gp/T/ipykernel_5197/3016704985.py:4: DtypeWarning: Columns (7,8,16,17,18,19,20,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_auto_loan_test = pd.read_csv('Test_Dataset.csv')


,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Client_Income_Type,Client_Education,Client_Marital_Status,Loan_Contract_Type,Age_Days,Employed_Days,Default
0,12142509,6750,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Commercial,Secondary,M,CL,13957,1062,0.0
1,12138936,20250,1.0,0.0,1.0,None,0.0,15282,1826.55,Service,Graduation,M,CL,14162,4129,0.0
2,12181264,18000,0.0,0.0,1.0,0.0,1.0,59527.35,2788.2,Service,Graduation dropout,W,CL,16790,5102,0.0
3,12188929,15750,0.0,0.0,1.0,1.0,0.0,53870.4,2295.45,Retired,Secondary,M,CL,23195,365243,0.0
4,12133385,33750,1.0,0.0,1.0,0.0,2.0,133988.4,3547.35,Commercial,Secondary,M,CL,11366,2977,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80895,12210181,11700,0.0,0.0,0.0,1.0,2.0,107820,3152.25,None,Secondary,M,CL,13867,1165,None
80896,12206257,29250,0.0,1.0,1.0,1.0,2.0,116430.75,4630.5,Govt Job,Graduation,M,None,14020,1699,None
80897,12200446,54000,1.0,0.0,1.0,1.0,0.0,52128,2742.3,Service,Secondary,M,CL,9954,796,None
80898,12244943,13500,0.0,0.0,1.0,0.0,0.0,25470,1435.05,Service,Graduation,M,CL,21464,2335,None


In [4]:
#Drops any rows with na values in the columns we are selecting
df_auto_loan = df_auto_loan.dropna(subset=selected_columns)
df_auto_loan

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Client_Income_Type,Client_Education,Client_Marital_Status,Loan_Contract_Type,Age_Days,Employed_Days,Default
0,12142509,6750,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Commercial,Secondary,M,CL,13957,1062,0.0
2,12181264,18000,0.0,0.0,1.0,0.0,1.0,59527.35,2788.2,Service,Graduation dropout,W,CL,16790,5102,0.0
3,12188929,15750,0.0,0.0,1.0,1.0,0.0,53870.4,2295.45,Retired,Secondary,M,CL,23195,365243,0.0
4,12133385,33750,1.0,0.0,1.0,0.0,2.0,133988.4,3547.35,Commercial,Secondary,M,CL,11366,2977,0.0
5,12191614,11250,0.0,1.0,1.0,1.0,1.0,13752,653.85,Service,Secondary,W,CL,13881,1184,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121850,12136406,12150,0.0,0.0,1.0,0.0,0.0,78192,2383.65,Retired,Secondary,S,CL,23943.0,365243.0,0.0
121852,12173765,15750,0.0,1.0,1.0,0.0,0.0,104256,3388.05,Commercial,Graduation,M,CL,8648.0,636.0,0.0
121853,12103937,8100,0.0,1.0,0.0,1.0,1.0,55107.9,2989.35,Govt Job,Secondary,M,CL,9152.0,1623.0,0.0
121854,12170623,38250,1.0,1.0,0.0,1.0,0.0,45000,2719.35,Service,Graduation,M,CL,10290.0,847.0,0.0


In [5]:
for col in selected_columns:
    
    #Ensures that the values in the numeric columns are float types
    if col in numeric_columns:
        df_auto_loan[col] = [float(elem) for elem in list(df_auto_loan[col])]
    #Converts the Loan_Contract_Type to binary format, where Cash Loans = 0 and Revolving Loans = 1
    if col in binary_columns:
        if col == 'Loan_Contract_Type':
            df_auto_loan[col] = [0 if elem == 'CL' else 1 for elem in list(df_auto_loan[col])]
        else:
            df_auto_loan[col] = [int(elem) for elem in list(df_auto_loan[col])]

/var/folders/w5/srvyynsx0xl0xdcy3blxc3800000gp/T/ipykernel_5197/1453635072.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auto_loan[col] = [float(elem) for elem in list(df_auto_loan[col])]
/var/folders/w5/srvyynsx0xl0xdcy3blxc3800000gp/T/ipykernel_5197/1453635072.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auto_loan[col] = [int(elem) for elem in list(df_auto_loan[col])]
/var/folders/w5/srvyynsx0xl0xdcy3blxc3800000gp/T/ipykernel_5197/1453635072.py:9: SettingWithCopyWarning: 
A value is tr

In [6]:
#Remove impossible values (ex: when Employed Days > Age Days)
df_auto_loan = df_auto_loan[df_auto_loan['Employed_Days'] <= df_auto_loan['Age_Days']]
#Clean out data cases that are exceptions in the dataset
s_bool = [elem in ['Service', 'Commercial', 'Govt Job'] for elem in df_auto_loan['Client_Income_Type']]
df_auto_loan = df_auto_loan.loc[s_bool, :]
#Removes limited postgrad values from the dataset
s_bool = [elem in ['Secondary', 'Graduation', 'Graduation dropout', 'Junior secondary'] for elem in df_auto_loan['Client_Education']]
df_auto_loan = df_auto_loan.loc[s_bool, :]
#Renames the education values in the dataset
rename_education_values_dict = {'Secondary': 'High_School', 
                                'Graduation': 'College', 
                                'Graduation dropout': 'College_Dropout', 
                                'Junior secondary': 'Middle_School'}
df_auto_loan['Client_Education'] = [rename_education_values_dict[elem] for elem in df_auto_loan['Client_Education']]
df_auto_loan

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Client_Income_Type,Client_Education,Client_Marital_Status,Loan_Contract_Type,Age_Days,Employed_Days,Default
0,12142509,6750.0,0,0,1,0,0.0,61190.55,3416.85,Commercial,High_School,M,0,13957.0,1062.0,0
2,12181264,18000.0,0,0,1,0,1.0,59527.35,2788.20,Service,College_Dropout,W,0,16790.0,5102.0,0
4,12133385,33750.0,1,0,1,0,2.0,133988.40,3547.35,Commercial,High_School,M,0,11366.0,2977.0,0
5,12191614,11250.0,0,1,1,1,1.0,13752.00,653.85,Service,High_School,W,0,13881.0,1184.0,0
10,12100935,27000.0,0,0,1,1,1.0,67500.00,3375.00,Govt Job,College,S,1,14816.0,2263.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121849,12183464,10350.0,0,1,0,0,0.0,18792.90,1736.55,Service,College_Dropout,S,0,9204.0,763.0,0
121852,12173765,15750.0,0,1,1,0,0.0,104256.00,3388.05,Commercial,College,M,0,8648.0,636.0,0
121853,12103937,8100.0,0,1,0,1,1.0,55107.90,2989.35,Govt Job,High_School,M,0,9152.0,1623.0,0
121854,12170623,38250.0,1,1,0,1,0.0,45000.00,2719.35,Service,College,M,0,10290.0,847.0,0


In [7]:
#Splits the dataset into all combinations of numeric, binary, and categorical (which we will investigate later)

df_auto_loan_numeric = df_auto_loan[id_column + numeric_columns + ['Default']].set_index('ID')
df_auto_loan_binary = df_auto_loan[id_column + binary_columns].set_index('ID')
df_auto_loan_numeric_binary = df_auto_loan[id_column + numeric_columns + binary_columns].set_index('ID')
df_auto_loan_numeric_binary_categorical = pd.get_dummies(df_auto_loan, columns=categorical_columns, drop_first=True).set_index('ID')
df_auto_loan_binary_categorical = df_auto_loan_numeric_binary_categorical.loc[:, [col for col in list(df_auto_loan_numeric_binary_categorical.columns) if col not in numeric_columns]]
df_auto_loan_numeric_categorical = df_auto_loan_numeric_binary_categorical.loc[:, ['Default'] + [col for col in list(df_auto_loan_numeric_binary_categorical.columns) if col not in binary_columns]]
df_auto_loan_categorical = df_auto_loan_numeric_binary_categorical.loc[:, ['Default'] + [col for col in list(df_auto_loan_numeric_binary_categorical.columns) if col not in binary_columns + numeric_columns]]


df_auto_loan_numeric_binary_categorical


,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Loan_Contract_Type,Age_Days,Employed_Days,Default,Client_Income_Type_Govt Job,Client_Income_Type_Service,Client_Education_College_Dropout,Client_Education_High_School,Client_Education_Middle_School,Client_Marital_Status_M,Client_Marital_Status_S,Client_Marital_Status_W
ID,,,,,,,,,,,,,,,,,,,,
12142509,6750.0,0,0,1,0,0.0,61190.55,3416.85,0,13957.0,1062.0,0,0,0,0,1,0,1,0,0
12181264,18000.0,0,0,1,0,1.0,59527.35,2788.20,0,16790.0,5102.0,0,0,1,1,0,0,0,0,1
12133385,33750.0,1,0,1,0,2.0,133988.40,3547.35,0,11366.0,2977.0,0,0,0,0,1,0,1,0,0
12191614,11250.0,0,1,1,1,1.0,13752.00,653.85,0,13881.0,1184.0,0,0,1,0,1,0,0,0,1
12100935,27000.0,0,0,1,1,1.0,67500.00,3375.00,1,14816.0,2263.0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12183464,10350.0,0,1,0,0,0.0,18792.90,1736.55,0,9204.0,763.0,0,0,1,1,0,0,0,1,0
12173765,15750.0,0,1,1,0,0.0,104256.00,3388.05,0,8648.0,636.0,0,0,0,0,0,0,1,0,0
12103937,8100.0,0,1,0,1,1.0,55107.90,2989.35,0,9152.0,1623.0,0,1,0,0,1,0,1,0,0


In [8]:
df_auto_loan_numeric.to_csv('df_auto_loan_numeric.csv')
df_auto_loan_binary.to_csv('df_auto_loan_binary.csv')
df_auto_loan_categorical.to_csv('df_auto_loan_categorical.csv')
df_auto_loan_numeric_binary.to_csv('df_auto_loan_numeric_binary.csv')
df_auto_loan_numeric_categorical.to_csv('df_auto_loan_numeric_categorical.csv')
df_auto_loan_binary_categorical.to_csv('df_auto_loan_binary_categorical.csv')
df_auto_loan_numeric_binary_categorical.to_csv('df_auto_loan_numeric_binary_categorical.csv')

In [9]:
#Standardizing the datasets:
#Note: We don't need to standardize the binary and categorical columns due to their format

df_auto_loan_numeric_std = df_auto_loan_numeric.copy()
df_auto_loan_numeric_binary_std = df_auto_loan_numeric_binary.copy()
df_auto_loan_numeric_binary_categorical_std = df_auto_loan_numeric_binary_categorical.copy()
df_auto_loan_numeric_categorical_std = df_auto_loan_numeric_categorical.copy()

for col in numeric_columns:
    
    col_mean = df_auto_loan_numeric_std[col].mean()
    col_std = df_auto_loan_numeric_std[col].std()
    df_auto_loan_numeric_std[col] = (df_auto_loan_numeric_std[col] - col_mean) / col_std
    
    col_mean = df_auto_loan_numeric_binary_std[col].mean()
    col_std = df_auto_loan_numeric_binary_std[col].std()
    df_auto_loan_numeric_binary_std[col] = (df_auto_loan_numeric_binary_std[col] - col_mean) / col_std
    
    col_mean = df_auto_loan_numeric_binary_categorical_std[col].mean()
    col_std = df_auto_loan_numeric_binary_categorical_std[col].std()
    df_auto_loan_numeric_binary_categorical_std[col] = (df_auto_loan_numeric_binary_categorical_std[col] - col_mean) / col_std
    
    col_mean = df_auto_loan_numeric_categorical_std[col].mean()
    col_std = df_auto_loan_numeric_categorical_std[col].std()
    df_auto_loan_numeric_categorical_std[col] = (df_auto_loan_numeric_categorical_std[col] - col_mean) / col_std
    

In [10]:
#Normalizing the datasets:
#Note: We don't need to normalize the binary and categorical columns due to their format

df_auto_loan_numeric_norm = df_auto_loan_numeric.copy()
df_auto_loan_numeric_binary_norm = df_auto_loan_numeric_binary.copy()
df_auto_loan_numeric_binary_categorical_norm = df_auto_loan_numeric_binary_categorical.copy()
df_auto_loan_numeric_categorical_norm = df_auto_loan_numeric_categorical.copy()

for col in numeric_columns:
    
    col_min = df_auto_loan_numeric_norm[col].min()
    col_max = df_auto_loan_numeric_norm[col].max()
    df_auto_loan_numeric_norm[col] = (df_auto_loan_numeric_norm[col] - col_min) / (col_max - col_min)
    
    col_min = df_auto_loan_numeric_binary_norm[col].min()
    col_max = df_auto_loan_numeric_binary_norm[col].max()
    df_auto_loan_numeric_binary_norm[col] = (df_auto_loan_numeric_binary_norm[col] - col_min) / (col_max - col_min)
    
    col_min = df_auto_loan_numeric_binary_categorical_norm[col].min()
    col_max = df_auto_loan_numeric_binary_categorical_norm[col].max()
    df_auto_loan_numeric_binary_categorical_norm[col] = (df_auto_loan_numeric_binary_categorical_norm[col] - col_min) / (col_max - col_min)
    
    col_min = df_auto_loan_numeric_categorical_norm[col].min()
    col_max = df_auto_loan_numeric_categorical_norm[col].max()
    df_auto_loan_numeric_categorical_norm[col] = (df_auto_loan_numeric_categorical_norm[col] - col_min) / (col_max - col_min)
    

In [11]:
df_auto_loan_numeric_std.to_csv('df_auto_loan_numeric_std.csv')
df_auto_loan_binary.to_csv('df_auto_loan_binary.csv')
df_auto_loan_categorical.to_csv('df_auto_loan_categorical.csv')
df_auto_loan_numeric_binary_std.to_csv('df_auto_loan_numeric_binary_std.csv')
df_auto_loan_numeric_categorical_std.to_csv('df_auto_loan_numeric_categorical_std.csv')
df_auto_loan_binary_categorical.to_csv('df_auto_loan_binary_categorical.csv')
df_auto_loan_numeric_binary_categorical_std.to_csv('df_auto_loan_numeric_binary_categorical_std.csv')

In [12]:
df_auto_loan_numeric_norm.to_csv('df_auto_loan_numeric_norm.csv')
df_auto_loan_binary.to_csv('df_auto_loan_binary.csv')
df_auto_loan_categorical.to_csv('df_auto_loan_categorical.csv')
df_auto_loan_numeric_binary_norm.to_csv('df_auto_loan_numeric_binary_norm.csv')
df_auto_loan_numeric_categorical_norm.to_csv('df_auto_loan_numeric_categorical_norm.csv')
df_auto_loan_binary_categorical.to_csv('df_auto_loan_binary_categorical.csv')
df_auto_loan_numeric_binary_categorical_norm.to_csv('df_auto_loan_numeric_binary_categorical_norm.csv')